In [1]:
import sys
import os
import transformers
import json
import torch
import numpy as np
from torch.utils.data import DataLoader , Dataset

/mnt/DGX01/Personal/slndir/evaluation/lib/python3.12/site-packages/transformers/utils/hub.py:127: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [2]:
sys.path.append("../")

In [3]:
#loading in the JSON list of dictionaries
data = json.load(open("../data/geneset_dict.json"))

In [4]:
for item in data:
    print(item)
    break

{'Symbol': 'A1BG', 'Gene_ID': 'ENSG00000121410', 'Name': 'alpha-1-B glycoprotein', 'Aliases': ['A1B', 'ABG', 'GAB', 'HYST2477'], 'Other names': ['HEL-S-163pA', 'alpha-1B-glycoprotein', 'epididymis secretory sperm binding protein Li 163pA'], 'Description': 'The protein encoded by this gene is a plasma glycoprotein of unknown function. The protein shows sequence similarity to the variable regions of some immunoglobulin supergene family member proteins. [provided by RefSeq, Jul 2008].'}


In [5]:
sampler = DataLoader(data, batch_size=1, shuffle=True)
sample = next(iter(sampler))

In [6]:
print(sample)

{'Symbol': ['GPC1'], 'Gene_ID': ['ENSG00000063660'], 'Name': ['glypican 1'], 'Aliases': ['glypican'], 'Other names': [('epididymis secretory sperm binding protein',), ('glypican proteoglycan 1',), ('glypican-1',)], 'Description': ['Cell surface heparan sulfate proteoglycans are composed of a membrane-associated protein core substituted with a variable number of heparan sulfate chains. Members of the glypican-related integral membrane proteoglycan family (GRIPS) contain a core protein anchored to the cytoplasmic membrane via a glycosyl phosphatidylinositol linkage.  These proteins may play a role in the control of cell division and growth regulation. [provided by RefSeq, Jul 2008].']}


In [7]:
query = """
Before proceeding with this task, please ask me any questions you need to provide the most helpful response possible. Consider aspects like context, specific requirements, format preferences, and any constraints I should be aware of.
Format this data into several question and answer pairs. Use the information from the keys to make multiple questions, and the information from the values to make the answer. Format multiple questions to resemble a user query. Make sure you use every key for its own separate question, and number the questions. Adress every key, but ignore any objects with ['N/A'] as their value.
Provide the pair in this format: <question>question here</question><answer>answer here</answer>

"""

In [8]:
print(query + str(sample))


Before proceeding with this task, please ask me any questions you need to provide the most helpful response possible. Consider aspects like context, specific requirements, format preferences, and any constraints I should be aware of.
Format this data into several question and answer pairs. Use the information from the keys to make multiple questions, and the information from the values to make the answer. Format multiple questions to resemble a user query. Make sure you use every key for its own separate question, and number the questions. Adress every key, but ignore any objects with ['N/A'] as their value.
Provide the pair in this format: <question>question here</question><answer>answer here</answer>

{'Symbol': ['GPC1'], 'Gene_ID': ['ENSG00000063660'], 'Name': ['glypican 1'], 'Aliases': ['glypican'], 'Other names': [('epididymis secretory sperm binding protein',), ('glypican proteoglycan 1',), ('glypican-1',)], 'Description': ['Cell surface heparan sulfate proteoglycans are compose

In [9]:
#load in llama model
from transformers import pipeline

model_id = "meta-llama/Llama-3.1-70B-Instruct"

pipe = pipeline(
    "text-generation",
    model=model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)
messages = [
    {"role": "system", "content": query},
    {"role": "user", "content": str(sample)},
]
outputs = pipe(
    messages,
    max_new_tokens=256,
)
print(outputs[0]["generated_text"][-1])

Loading checkpoint shards:   0%|          | 0/30 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


{'role': 'assistant', 'content': "To provide the most helpful response, I need to confirm a few details. \n\n1. You would like me to create question and answer pairs based on the given dictionary data.\n2. I should use the keys as the basis for the questions and the corresponding values as the answers.\n3. I should format multiple questions to resemble a user query when there are multiple values for a single key.\n4. I should ignore any objects with ['N/A'] as their value, but since there are no ['N/A'] values in the given data, this will not be applicable.\n5. You would like the pairs in the format: <question>question here</question><answer>answer here</answer>\n\nPlease confirm these details before I proceed.\n\nAlso, I would like to know if you would like me to make any adjustments to the answer for the 'Description' key, as it is a long piece of text. Would you like me to shorten it or keep it as is?"}
